## El dataset de LaLonde

Estos datos se han vuelto famosos para ejemplificar y entrenar lo difícil que puede ser entender los efectos. Provienen de un artículo escrito por Robert LaLonde en 1986, y que trata de entender lso efectos de un programa de entrenamiento laboral, un par de años después que ese entrenamiento concluyera. 

El dataset son solo hombres. Tiene una columna **treat**, por _treatment_, que marca con un 1 aquellas personas que si completaron el programa. **age** es la edad en años, **educ** son los años de educación de la persona, **black** e **hispan** se refiere a la raza, **married** al estado civil, **nodegree** asigna un 1 a quienes no tienen un grado universitario, **re74** y **re75** son los salarios reales de las personas en los años 1974 y 1975 (antes del programa), y **re78** son los salarios el año 1978, después de que terminara el programa. 


In [173]:
import pandas as pd

lalonde = pd.read_csv('lalonde.csv', index_col=0)
lalonde.head()

,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
id,,,,,,,,,,
NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899


## Primer análisis: medias (average treatment effect), test t para ver si dos series vienen de la misma distribución. 

Veamos primero la idea más obvia que se nos puede ocurrir: si queremos ver que el programa funciona, veamos el salario promedio de la gente que recibió el programa y los que no? 

In [174]:
lalonde.groupby("treat").mean()["re78"]

treat
0    6984.169742
1    6349.143530
Name: re78, dtype: float64

!El grupo sin entrenamiento obtiene mayor salario! ¿Será que el programa en realida dhace mal? O, tal vez, esta diferencia podría ser simplemente por cómo se distribuyeron los grupos. Para ver si es estadísticamente significativa, vamos a hacer un test t. 

In [175]:
from scipy.stats import ttest_ind

lalonde_t0 = lalonde[lalonde["treat"]==0]
lalonde_t1 = lalonde[lalonde["treat"]==1]

ttest_ind(lalonde_t0["re78"], lalonde_t1["re78"])


Ttest_indResult(statistic=0.9663522254463778, pvalue=0.3342496685909654)

Lo primero que retorna la función es el estadístico t, y lo segundo es el valor de significancia. A un 33%, no podemos deducir que los salarios son significativamente distintos, y por lo tanto, tal vez deberíamos concluír en vez que en realidad no hay diferencia entre los que toman el programa y los que no. 

Esto sigue siendo extraño, y vamos a tratar de hacer algo mejor. Contemos cuantos individuos hay por grupo. 

In [176]:
lalonde.groupby("treat").count()

,age,educ,black,hispan,married,nodegree,re74,re75,re78
treat,,,,,,,,,
0,429,429,429,429,429,429,429,429,429
1,185,185,185,185,185,185,185,185,185


Ahora cuantos hay con salario = 0 (es decir, que no tenían trabajo). Vemos que son más de la mitad de los que si recibieron el _treatment_ (el programa de entrenamiento laboral). Además, si consideramos los salarios el 74 y el 75, el grupo con _treatment_ 1 parte desde un salario mucho más bajo. 

In [177]:
lalonde[lalonde["re74"]<=0].groupby("treat").count()

,age,educ,black,hispan,married,nodegree,re74,re75,re78
treat,,,,,,,,,
0,112,112,112,112,112,112,112,112,112
1,131,131,131,131,131,131,131,131,131


In [178]:
lalonde.groupby("treat").mean()[["re74","re75","re78"]]

,re74,re75,re78
treat,,,
0,5619.236506,2466.484443,6984.169742
1,2095.573689,1532.055314,6349.143530


Por todo esto, ¿pareciera ser que el programa si funciona? Después de todo, los individuos del programa logran subir su salario mucho más que los individuos que no asisten al programa. 

Ahora, se podría decir también que no tenemos mucho fundamento para admitir esto, puesto que quizás las personas que asistieron al programa tenían de por sí un mayor potencial para subir su salario. Por supuesto, no podemos saber a ciencia cierta qué hubiese pasado si las personas que no tomaron el programa (_treatment_ = 0) en realidad lo toman. Pero podemos hacer varias cosas adicionales. 

Por ejemplo, podemos estimar una regresion lineal que tome en cuenta algunas variables, y que trate de predecir el salario del año 1978. Lo primero sería usar solo la variable **treat**


In [179]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

X = lalonde[["treat"]]
y = lalonde["re78"]

# Crea un objeto regresión
regr = linear_model.LinearRegression()
regr.fit(X,y)

print('Coeficientes: \n', regr.coef_)
print('Beta0: \n', regr.intercept_)

Coeficientes: 
 [-635.02621204]
Beta0: 
 6984.169742307693


Nos da que el efecto es negativo, es decir, que un incremento en **treat** debería contribuir de forma negativa al salario. Pero eso ya lo vimos cuando tomamos las medias (nota que la diferencia en las medias es exactamente lo mismo que el coeficiente de acá. ¿Puedes ver por qué?). 

Una cosa un poco más útil sería _controlar_ por algunas variables. Para esto, estimamos una regresión con algunos parámetros adicionales, en nuestro caso los salarios anteriores, y vemos el signo del coeficiente. 

In [180]:

X = lalonde[["treat","re74","re75"]]
y = lalonde["re78"]

# Crea un objeto regresión
regr = linear_model.LinearRegression()
regr.fit(X,y)

print('Coeficientes: \n', regr.coef_)
print('Beta0: \n', regr.intercept_)

Coeficientes: 
 [7.98127034e+02 3.48177080e-01 2.20770746e-01]
Beta0: 
 4483.152775135084


Ahora el signo es positivo: cuando los salarios se mantienen constantes, un incremento en **treat** produce un  incremento importante en el salario. Ahora bien, podría ser que los salarios también dependieran de algo más, por ejemplo, es razonable pensar que los hombres con salarios bajos son más propensos para tomar el programa de entrenamiento. 

## Matching y propensity score

De forma un poco más avanzada, uno puede intentar _simular_ que los grupos **treat**=0 y **treat**=1 hayan sido dispuestos de forma aleatoria. En esta actividad vamos a ver una forma de hacerlo: vamos a hacerles matching y luego estratificar en cinco quintiles, usando lo que se conoce como el __propensity score__. 

Lo primero que vamos a hacer es usar dos clasificadores: regresión logística y random forest, para poder predecir que tan probable era que una persona recibiera el _treatment_, en este caso, el programa de entrenamiento laboral. 

In [181]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

logistic = linear_model.LogisticRegression(max_iter=500)

X = lalonde[["age", "educ", "black", "hispan", "married", "nodegree", "re74", "re75"]]
y = lalonde["treat"]


precision = cross_val_score(logistic, X, y, cv = 10, scoring = "precision")
recall = cross_val_score(logistic, X, y, cv = 10, scoring = "recall")

print("precision: "+str(precision.mean()))
print("recall: "+str(recall.mean()))


precision: 0.6265300596376447
recall: 0.6461988304093567


In [182]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier(max_depth = 2)


precision = cross_val_score(rforest, X, y, cv =10, scoring = "precision")
recall = cross_val_score(rforest, X, y, cv = 10, scoring = "recall")

print("precision: "+str(precision.mean()))
print("recall: "+str(recall.mean()))



C:\Users\diego\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision: 0.6246031746031746
recall: 0.4625730994152046


## Ejercicio 1

¿Cuál de los clasificadores parece el mejor? Valdrá la pena probar con un KNN y ver que pasa?

Como se puede ver, la regresion logistica presenta valores mas altos, tanto para precision como para recall. Asi, el modelo de regresion logistica se adecua mas, logrando menos falsos positivos y negativos que el modelo random forest. Respecto a utilizar un modelo KNN, el hecho de que haya una mayor cantidad de datos treat=0 significa que el modelo tenderia a asignar 0 a valores cercanos. La proporcion entre valores 1 y 0 debe ser idealmente 50/50, por lo que no valdria la pena realizar un test con este.

Ahora vamos a usar predict_proba para pedir la probabilidad de que una persona tenga el _treatment_ (osea que su variable treatment sea 1). Esta probabilidad es el propensity score. 

In [183]:
logistic.fit(X,y) # entrenamos el modelo ahora si con todos los datos

prediction = logistic.predict_proba(X) 

# ahora le agregamos a nuestro dataframe el score. 

lalonde['propensity_score'] = pd.Series(prediction[:,1], index = lalonde.index)
lalonde.head()

,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78,propensity_score
id,,,,,,,,,,,
NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460,0.431932
NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940,0.134968
NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500,0.719753
NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460,0.658953
NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899,0.695676


Ahora vamos a dividir nuestro dataset en cinco quintiles, de acuerdo al propensity score. Esto va a significar poder agrupar por individuos que se parezcan más o menos en cuanto a qué tan probable era que recibieran el _treatment_. 

In [184]:
quintiles = lalonde['propensity_score'].quantile([0, .2, .4, .6, .8, 1])
quintiles

0.0    0.009155
0.2    0.061409
0.4    0.126959
0.6    0.197364
0.8    0.675369
1.0    0.781233
Name: propensity_score, dtype: float64

In [185]:
lalondeq1 = lalonde[lalonde["propensity_score"]<0.060138]
lalondeq2 = lalonde[(lalonde["propensity_score"]>=0.060138) & (lalonde["propensity_score"]<0.126356)]
lalondeq3 = lalonde[(lalonde["propensity_score"]>=0.126356) & (lalonde["propensity_score"]<0.198206)]
lalondeq4 = lalonde[(lalonde["propensity_score"]>=0.198206) & (lalonde["propensity_score"]<0.677507)]
lalondeq5 = lalonde[lalonde["propensity_score"]>=0.677507]

## Ejercicio 2

Mira estos cinco quintiles. ¿En cuales la media del ingreso en el año 78 es más grande para el grupo _treatment_ = 1? ¿Hay alguno en que todavía sea más grande el de _treatment_ = 0? ¿Por qué crees que es eso? 


In [188]:
print(lalondeq1.groupby("treat").mean()[["re74","re78","black","hispan","age","married"]])
print(lalondeq2.groupby("treat").mean()[["re74","re78","black","hispan","age","married"]])
print(lalondeq3.groupby("treat").mean()[["re74","re78","black","hispan","age","married"]])
print(lalondeq4.groupby("treat").mean()[["re74","re78","black","hispan","age","married"]])
print(lalondeq5.groupby("treat").mean()[["re74","re78","black","hispan","age","married"]])

               re74          re78  black    hispan        age  married
treat                                                                 
0      13304.738496  10787.180487    0.0  0.109244  32.722689  0.97479
1       7403.025500   3394.231500    0.0  0.500000  23.000000  1.00000
              re74         re78     black    hispan        age   married
treat                                                                   
0      3827.210997  5880.998505  0.000000  0.247788  30.743363  0.690265
1      6761.894611  9284.531133  0.111111  0.555556  23.222222  0.555556
              re74         re78  black    hispan        age   married
treat                                                                
0      1144.028680  5523.907675    0.0  0.186916  21.803738  0.009346
1       572.422968  8611.749421    0.0  0.263158  25.947368  0.000000
              re74         re78     black  hispan        age   married
treat                                                                 
0 

El unico quintil donde la media de ingreso para el año 78 es mayor en gente que no realizó el programa es el primero. Para el resto de quintiles, la media es mayor para treat = 1.
La razon de esto es evidente dado las caracteristicas de este grupo(primer quintil). Vemos que los que no realizaron el programa en el primer quintil tienen un sueldo promedio muy alto para el año 74, ademas de tener el promedio de edad mas alto entre todos los grupos, y con la mayoria casados. Ademas, casi ninguna persona en el quintil 1 que no hizo el programa se identifica como afroamericano o hispanico, fragmentos de la sociedad historicamente oprimidos en Estados Unidos, y quienes necesitarian principalmente programas que les ayuden a surgir. Asi, podemos deducir que este grupo corresponde a gente de edad mayor, blancos, de clase alta en promedio. Asi, es muy poco probable que necesiten realizar el programa, no como el quintil 5, que esta compuesto casi totalmente por gente afroamericana joven.

## Ejercicio 3

Ahora calcula estadísticos t para cada quintil. Cuenta, además, cuantos indviduos hay por quintil. ¿Hay algo que podamos decir con propiedad? 


In [194]:
lalondeq1_t0 = lalondeq1[lalondeq1["treat"]==0]
lalondeq1_t1 = lalondeq1[lalondeq1["treat"]==1]
print(ttest_ind(lalondeq1_t0["re78"], lalondeq1_t1["re78"]))
lalondeq2_t0 = lalondeq2[lalondeq2["treat"]==0]
lalondeq2_t1 = lalondeq2[lalondeq2["treat"]==1]
print(ttest_ind(lalondeq2_t0["re78"], lalondeq2_t1["re78"]))
lalondeq3_t0 = lalondeq3[lalondeq3["treat"]==0]
lalondeq3_t1 = lalondeq3[lalondeq3["treat"]==1]
print(ttest_ind(lalondeq3_t0["re78"], lalondeq3_t1["re78"]))
lalondeq4_t0 = lalondeq4[lalondeq4["treat"]==0]
lalondeq4_t1 = lalondeq4[lalondeq4["treat"]==1]
print(ttest_ind(lalondeq4_t0["re78"], lalondeq4_t1["re78"]))
lalondeq5_t0 = lalondeq5[lalondeq5["treat"]==0]
lalondeq5_t1 = lalondeq5[lalondeq5["treat"]==1]
print(ttest_ind(lalondeq5_t0["re78"], lalondeq5_t1["re78"]))

Ttest_indResult(statistic=1.1958514331011507, pvalue=0.23413198224233306)
Ttest_indResult(statistic=-1.4124918589916236, pvalue=0.16039341570298762)
Ttest_indResult(statistic=-2.1951052008800143, pvalue=0.030017999776553003)
Ttest_indResult(statistic=-0.13776412870548888, pvalue=0.8906539495909104)
Ttest_indResult(statistic=-1.2579218253938593, pvalue=0.21088278480315145)


In [196]:
print(len(lalondeq1))
print(len(lalondeq2))
print(len(lalondeq3))
print(len(lalondeq4))
print(len(lalondeq5))

121
122
126
124
121


Para el quintil 3 vemos un p-value menor a 0.05. Esto nos dice que para los 126 individuos en este quintil, el realizar el programa genera una diferencia estadisticamente  significante respecto a su salario. Es decir, para este grupo de personas, hay evidencia de que haber realizado el programa se correlaciona con obtener un salario mas alto en el año 1978. Para todo el resto de quintiles el p-value es demasiado alto como para poder concluir algo.